*Note that most of the code cells are hidden from this notebook for a better reading flow. Check out the notebook in the documentation folder in the Github repository for more details.*

# Replicating Keane and Wolpin (1994)

Keane and Wolpin (1994) and the previously published working paper Keane and Wolpin (1994b) generate three different Monte Carlo samples. This notebook replicates some of their results as well as giving other insights into the model.

## Returns to experience

We first plot the returns to experience while holding education constant at the initial ten years. Occupation B is more skill intensive in the sense that own experience has higher return than is the case for Occupation A. There is some general skill learned in Occupation A which is transferable to Occupation B. However, work experience is occupation-specific in Occupation B.

In [1]:
%matplotlib agg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import respy as rp
import warnings

from mpl_toolkits.mplot3d import Axes3D

In [2]:
plt.style.use("notebook_material/respy_static/respy.mplstyle")

## Effect of college tuition subsidy

This section replicates Table 6 in Keane and Wolpin (1994) which studies the effect of different amounts of tuition subsidies. First, we are going to show the effect of the tuition subsidy on each of the parametrizations for a sample of 1,000 simulated individuals. After that, Table 6 is replicated where the policy effect is measured as the average difference in experience for 40 samples with 100 invididuals with and without the tutition subsidy. The authors apply a 500 USD tuition subsidy on the first parametrization, 1000 USD on the second and 2000 USD on the third parametrization.

The following figures show the impact of the tuition subsidies for each of the parametrizations next to each other.

In [3]:
# Build simulate function as only parameters change, it can be reused.
params, options = rp.get_example_model("kw_94_one", with_data=False)
options["simulation_agents"] = 4000

In [4]:
simulate = rp.get_simulate_func(params, options)

models = np.repeat(["one"], 2)
tuition_subsidies = [0, 500]

data_frames = []

for model, tuition_subsidy in zip(models, tuition_subsidies):
    params, _ = rp.get_example_model(f"kw_94_{model}", with_data=False)
    params.loc[("nonpec_edu", "hs_graduate"), "value"] += tuition_subsidy
    data_frames.append(simulate(params))

In [5]:
for df in data_frames:
    df["Bootstrap_Sample"] = pd.cut(df.Identifier, bins=40, labels=np.arange(1, 41))

In [6]:
df_wo_ts = data_frames[0]
df_w_ts = data_frames[1]

# Split the sample in 40 parts.
df_wo_ts["Bootstrap_Sample"] = pd.cut(df_wo_ts.Identifier, bins=40, labels=np.arange(1, 41))
df_w_ts["Bootstrap_Sample"] = pd.cut(df_w_ts.Identifier, bins=40, labels=np.arange(1, 41))

mean_exp_wo_ts = df_wo_ts.loc[df_wo_ts.Period.eq(39), ["Bootstrap_Sample", "Experience_Edu"]].groupby("Bootstrap_Sample").mean()
mean_exp_w_ts = df_w_ts.loc[df_w_ts.Period.eq(39), ["Bootstrap_Sample", "Experience_Edu"]].groupby("Bootstrap_Sample").mean()

diff = mean_exp_w_ts - mean_exp_wo_ts
diff = diff.reset_index().set_index(["Bootstrap_Sample"]).stack()

In [7]:
qoi = diff.mean()
qoi

1.5417500000000004

## References

> Keane, M. P. and  Wolpin, K. I. (1994). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://doi.org/10.2307/2109768). *The Review of Economics and Statistics*, 76(4): 648-672.

> Keane, M. P. and  Wolpin, K. I. (1994b). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://www.minneapolisfed.org/research/staff-reports/the-solution-and-estimation-of-discrete-choice-dynamic-programming-models-by-simulation-and-interpolation-monte-carlo-evidence). *Federal Reserve Bank of Minneapolis*, No. 181.